In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
import US_main, US_c, US_policy
os.chdir(d['curr'])

## US FH:

Define data and parameters:

In [2]:
dates_log = pd.Index([1950, 1980, 2010, 2040, 2070, 2100], name = 't')
ν_log = np.array([1.504840069, 1.394563144, 1.178137696, 1.055068673, 1.018706685, 1.018706685])
T_log = len(ν_log)
T = T_log+3
dates = dates_log.union(pd.Index([dates_log[-1]+30*i for i in range(1,T-T_log+1)]))
ν = np.hstack([ν_log, np.full(T-T_log, ν_log[-1])])
A = np.ones(T) # normalize exog. productivity at 1 
t0date = 2010 # calibration date
t0 = dates.get_loc(t0date) # index for year of calibration 


# "Simple" calibration:
α = .281 # capital income share

# Household types:
γj  = np.array([.056, 0.449, 0.307, 0.188])
ni = len(γj)-1
hours = np.array([1415.38, 2114.79, 2315.83, 2458.91]) # hours
wagerates = np.array([6.914, 14.88, 27.32, 62.26]) # wages
income = hours*wagerates
zxj = hours/hours[1:].mean()
zηj = income/income[1:].mean()
pj = np.array([0.385, 0.55, 0.652, 0.74])
μj = np.array([0.362, 0.526, 0.684, 0.756]) # voter turnout 


# other targets:
τ0 = .158 # target labor tax rate
RR = 39.4/50.1 # replacement rate
universalShare = 3.4/15.8 # share of ss going to universal
R0 = 2.443 


# Initial/baseline values:
ξ0 = .1
ρ0 = 1.1
ω0 = 1.5
# βj = np.ones(ni+1)

Package them in kwargs:

In [3]:
kwargs = {'α': α, 'A': A, 'ν': ν, 'γj': γj, 'zxj': zxj, 'zηj': zηj, 'pj': pj, 'μj': μj, 'τ0': τ0, 'RR0': RR, 'UShare0': universalShare, 'R0': R0, 'ξ': ξ0, 'ρ': ρ0, 'ω': ω0}

Initialize model:

In [4]:
m = US_main.Model(ni = ni, T = T, gridkwargs = {'glob_ns': 50}, **kwargs)

### Calibrate to PEE - vary $\epsilon, \theta$

In [5]:
m.USCalSimple_PEE_FH(t0);

Get baseline solution:

In [6]:
sol, pol = m.PEE_FH(returnPols = True)

Solve on grid of epsilon/theta:

In [21]:
epsGrid, θGrid = pd.Index(np.round(np.linspace(0, 1, 11), 1), name = 'eps'), pd.Index(np.round(np.linspace(0,1,11),1), name = 'θ')
epsθGrid = pd.MultiIndex.from_product([epsGrid, θGrid])

Solve on grid:

In [33]:
db0 = m.db.copy()
solGrid, polGrid = dict.fromkeys(epsθGrid), dict.fromkeys(epsθGrid)

In [34]:
for (ϵ,θ) in epsθGrid:
    m.db.update(m.initSC(ϵ, 'eps'))
    m.db.update(m.initSC(θ, 'θ'))
    solGrid[(ϵ,θ)], polGrid[(ϵ,θ)] = m.PEE_FH(returnPols = True)

For a value of $\theta$, plot taxes as a function of $\epsilon$:

In [60]:
dfϵ = pd.concat([solGrid[(ϵ, .5)]['τ'] for ϵ in epsGrid], axis = 1)
dfθ = pd.concat([solGrid[(.5, θ)]['τ'] for θ in θGrid], axis = 1)